<a href="https://colab.research.google.com/github/Grashch/Kaggle_tasks/blob/main/Pipelines_home_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [16]:
# Загрузка обучающих и тестовых данных
# Индекс устанавливается по столбцу 'Id'
train_full = pd.read_csv('/content/train_pipelines_hw.csv', index_col='Id')
test_full = pd.read_csv('/content/test_pipelines_hw.csv', index_col='Id')

# Удаление строк с пропущенными значениями в целевом столбце 'SalePrice'
train_full.dropna(subset=['SalePrice'], axis=0, inplace=True)

# Выделение целевой переменной и признаков
y = train_full['SalePrice']
X = train_full.drop('SalePrice', axis=1)


In [43]:
# Отбор категориальных столбцов:
# - тип данных — строка (object)
# - количество уникальных значений < 10
c_cols = [
    col for col in X
    if X[col].dtype == 'object' and X[col].nunique() < 10
]

# Отбор числовых столбцов:
# - тип данных — int64 или float64
n_cols = [
    col for col in X
    if X[col].dtype in ['int64', 'float64']
]

# Объединение отобранных столбцов
my_cols = c_cols + n_cols

# Ограничение набора признаков только отобранными столбцами
X = X[my_cols]

In [42]:
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и тестовую выборки
# test_size=0.3 — 30% данных для тестирования
# random_state=3 — фиксируем случайность для воспроизводимости
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=3
)

# Проверяем разверности split-данных
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [26]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [33]:
# Создание трансформера для числовых столбцов
# Заменяет пропуски медианой столбца
n_cols_transformer = SimpleImputer(strategy='median')

# Создание пайплайна для категориальных столбцов
# 1. Заменяет пропуски наиболее часто встречающимся значением
# 2. Применяет one-hot-кодирование
# handle_unknown='ignore' — игнорирует новые категории в тестовых данных
c_cols_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Объединение трансформеров в единый ColumnTransformer
# 'num' — имя блока для числовых данных
# 'cat' — имя блока для категориальных данных
transformer = ColumnTransformer([
    ('num', n_cols_transformer, n_cols),
    ('cat', c_cols_transformer, c_cols)
])

In [34]:
from sklearn.ensemble import RandomForestRegressor

# Создание модели случайного леса
# n_estimators=100 — количество деревьев в ансамбле
# random_state=3 — фиксируем случайность
model = RandomForestRegressor(n_estimators=100, random_state=3)

In [35]:
# Сборка полного пайплайна:
# 1. transformer — предобработка данных
# 2. modeling — обучение модели
my_pipeline = Pipeline(steps=[
    ('transform', transformer),
    ('modeling', model)
])

In [36]:
from sklearn.metrics import mean_absolute_percentage_error

python
from sklearn.metrics import mean_absolute_percentage_error

# Обучение пайплайна на обучающей выборке
my_pipeline.fit(X_train, y_train)

# Получение предсказаний на тестовой выборке
y_pred = my_pipeline.predict(X_test)

# Расчет метрики MAPE (средняя абсолютная процентная ошибка)
score = mean_absolute_percentage_error(y_test, y_pred)

# Вывод результата
print(f'MAPE: {score:.4f}')  # Форматирование до 4 знаков после запятой

MAPE: 0.09780924873036904


In [45]:
# Подготовка тестовых данных для предсказания
# Выбираем только нужные столбцы (как в обучающей выборке)
test_data = test_full[my_cols]


# Получение предсказаний для тестового набора данных
y_pred_test_date = my_pipeline.predict(test_data)

# Добавление столбца с предсказаниями в тестовый датафрейм
test_data['SalePricePredict'] = y_pred_test_date

# Просмотр первых 5 строк результата
test_data.head()

/tmp/ipython-input-3620403288.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['SalePricePredict'] = y_pred_test_date


,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePricePredict
Id,,,,,,,,,,,,,,,,,,,,,
1461,RH,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Feedr,Norm,...,140,0,0,0,120,0,0,6,2010,127546.33
1462,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,393,36,0,0,0,0,12500,6,2010,158395.50
1463,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,212,34,0,0,0,0,0,3,2010,181976.93
1464,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,360,36,0,0,0,0,0,6,2010,181114.50
1465,RL,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,0,82,0,0,144,0,0,1,2010,196217.87
